In [4]:
from matplotlib import pyplot
import statsmodels as sm
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_model import ARIMA
import statistics
import matplotlib as mpl
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np
from datetime import timedelta,date
pyplot.rcParams.update({'figure.figsize': (10, 7), 'figure.dpi': 120})
from sklearn.metrics import r2_score

# Import as Dataframe
# df = pd.read_csv('https://raw.githubusercontent.com/selva86/datasets/master/a10.csv', parse_dates=['date'])
with open('input\input01.txt', 'r') as file:
    data = file.read().split("\n")
    
n = int(data.pop(0))
listDate = []
for i in range(0,n):
    listDate.append(date(2012, 10, 1) + timedelta(i))
    
df = pd.DataFrame()
df['Date'] = listDate
df['traffic'] = list(map(int, data))
df = df.set_index(['Date'])
df.to_csv('traffic.csv') 
df.head()

,traffic
Date,
2012-10-01,1339
2012-10-02,1462
2012-10-03,1702
2012-10-04,1656
2012-10-05,1439


In [ ]:
from statsmodels.tsa.stattools import adfuller

def plot_df(x, y, title="", xlabel='Date', ylabel='Value', dpi=100):
    plt.figure(figsize=(16,5), dpi=dpi)
    plt.plot(x, y, color='tab:red')
    plt.gca().set(title=title, xlabel=xlabel, ylabel=ylabel)
    plt.show()
    
def testADF(data):
    print(" > Is the data stationary ?")
    dftest = adfuller(data, autolag='AIC')
    print("Test statistic = {:.3f}".format(dftest[0]))
    print("P-value = {:.3f}".format(dftest[1]))
    print("Critical values :")
    for k, v in dftest[4].items():
        print("\t{}: {} - The data is {} stationary with {}% confidence".format(k, v, "not" if v<dftest[0] else "", 100-int(k[:-1])))

plot_df(df['Date'].values, df['traffic'].values) 
testADF(df['traffic'].diff().dropna())

In [ ]:
data = df.copy()

In [ ]:
data = data.set_index(['Date'])
data

In [ ]:
window_size = 12
rolmean=data.rolling(window=window_size).mean()
rolstd=data.rolling(window=10).std()

In [ ]:
actual=plt.plot(data, color='red', label='Actual')
mean_6=plt.plot(rolmean, color='green', label='Rolling Mean') 
std_6=plt.plot(rolstd, color='black', label='Rolling Std')
plt.legend(loc='best')
plt.title('Rolling Mean & Standard Deviation')
plt.show(block=True)

In [ ]:
testADF(data)

In [ ]:
plt.figure(figsize=(20,10))
data_log=np.log(df['traffic'])
plt.plot(data_log)

In [ ]:
MAvg=data_log.rolling(window=12).mean()
MStd=data_log.rolling(window=12).std()
plt.plot(data_log)
plt.plot(MAvg, color='blue')

In [ ]:
data_log_diff=data_log-MAvg
data_log_diff.head(12)

In [ ]:
data_log_diff=data_log_diff.dropna()
data_log_diff.head()


In [ ]:
def stationarity(timeseries):
    
    rolmean=timeseries.rolling(window=12).mean()
    rolstd=timeseries.rolling(window=12).std()
    
    plt.figure(figsize=(20,10))
    actual=plt.plot(timeseries, color='red', label='Actual')
    mean_6=plt.plot(rolmean, color='green', label='Rolling Mean') 
    std_6=plt.plot(rolstd, color='black', label='Rolling Std')
    plt.legend(loc='best')
    plt.title('Rolling Mean & Standard Deviation')
    plt.show(block=False)
    

In [ ]:
stationarity(data_log_diff)
testADF(data_log_diff)

In [ ]:
plt.figure(figsize=(20,10))
data_shift=data_log-data_log.shift()
plt.plot(data_shift)

data_shift=data_shift.dropna()
stationarity(data_shift)

In [ ]:
from statsmodels.tsa.stattools import acf, pacf

lag_acf=acf(data_shift, nlags=20)
lag_pacf=pacf(data_shift, nlags=20, method='ols')

plt.figure(figsize=(20,10))
plt.subplot(121)
plt.plot(lag_acf)
plt.axhline(y=0,linestyle='--',color='green')
plt.axhline(y=-1.96/np.sqrt(len(data_shift)),linestyle='--',color='green')
plt.axhline(y=1.96/np.sqrt(len(data_shift)),linestyle='--',color='green')
plt.title('Autocorrelation Function')

plt.subplot(122)
plt.plot(lag_pacf)
plt.axhline(y=0,linestyle='--',color='green')
plt.axhline(y=-1.96/np.sqrt(len(data_shift)),linestyle='--',color='green')
plt.axhline(y=1.96/np.sqrt(len(data_shift)),linestyle='--',color='green')
plt.title('Partial Autocorrelation Function')

In [ ]:
data_shift = data_shift.dropna()
from statsmodels.tsa.arima_model import ARIMA

plt.figure(figsize=(20,10))
model=ARIMA(data_log, order=(1,1,1))
results=model.fit(disp=-1)
plt.plot(data_shift)
plt.plot(results.fittedvalues, color='red')
plt.title('RSS: %.4f'% sum((results.fittedvalues-data_shift['trafiic'])**2))
print('plotting ARIMA model')
